In [1]:
%config Completer.use_jedi = False

In [4]:
import numpy as np
import pandas as pd
from hazm import *
from collections import Counter

from typing import Tuple

In [5]:
df = pd.read_csv("base_responses.csv", encoding="utf-8").dropna()
df.head()

,Class1-Q,Class1-W,Class2-Q,Class2-W,Class3-Q,Class3-W,Class4-Q,Class4-W
0,آیا فردا هوا ابری است؟,آیا - هوا - فردا - ابری,چه زمانی اذان مغرب شروع می‌شود؟,زمان - اذان - مغرب,ساعت چنده؟,ساعت,میلاد پیامبر چه زمانی است؟,میلاد - پیامبر - زمان
1,هوا فردا آلودست ؟\nشاخص آلودگی چنده؟\nیعنی فرد...,برف - بارون - آلودگی - فردا - هواشناسی,اذان کیه؟\nطلوع آفتاب ساعت چنده؟\nاذان ظهرو گف...,اذان- غروب- افطار-طلوع-ظهر,الان تو مونیخ ساعت چنده؟\nاختلاف زمانیمون با ا...,شب - روز - ساعت - اختلاف زمانی - الان,امروز چندمه؟\nچند روز مونده تا عید؟\nوقت دکترت...,چند شنبه-تعطیلی-روز-چندمه؟-وقت
2,شب است.,شب,قبل از شب میباشد,قبل - شب,الان ۲۳ و ۲۳ دقیقه است.,۲۳ - ۲۳,چهارشنبه ۸ آذر,۴ - ۸ - ۹
3,آلودگی هوای تهران به چه علت است؟\nهوای نسبی ای...,هوا - تهران - ایران - دما - باران,اذان صبح چه ساعتی‌ست؟\nاختلاف زمانی اذان ظهر ش...,اذان - صبح - ظهر - تهران - مشهد - اختلاف,اختلاف زمانی ساعت ایران و ترکیه چقدر است؟\nامر...,اختلاف - ایران - ترکیه - امروز - سال - ۱۴۰۰,تاریخ وفات امام خمینی(ره) چه است؟\nتاریخ پیروز...,وفات - خمینی - تاریخ - شاه - خامنه‌ای - رهبر -...
5,امیدوارم هوای دیزین هفته آینده برفی باشد,دیزین-شمشک-دربندسر-برف-بارش-پودر-ابر-باران,اذان صبح خیلی زود است,اذان-طلوع-افتاب-موذن,در دنیای تو ساعت چند است,ساعت-زمان-شیراز-اصفهان-تهران-اختلاف زمانی,هفته دیگر,فردا-تعطیلی-امروز-دیروز-بین اتعطیلین-نوروز-کری...


In [6]:
unused_chars = ['-', '،', '_', '\n', '//', '/', '\u200c', 'ـ', '?', '؟']

In [37]:
words, combs = initialize()

In [38]:
COMB_SCORE = 6
WORD_SCORE = 2

In [39]:
def get_sw(filename: str) -> list:
    with open(filename, encoding="utf-8") as f:
        sw = f.read().split('\n')
        return sw

In [40]:
def get_list_from(filename: str) -> list:
    with open(filename) as f:
        temp = f.read().split('=')
        ret = []
        for t in temp:
            ret.append([w for w in t.split('\n') if w != ''])
        return ret

In [41]:
def parse_shamsi_events(sw: list) -> Tuple[list, list]:
    events = pd.read_csv("shamsi_events.csv", encoding="utf-8")
    combs = events["event"].tolist()
    words = []
    for c in combs:
        words += [w for w in norm.normalize(c).split(' ') if w not in sw]
    return list(words), combs

In [42]:
def initialize() -> Tuple[list, list]:
    sw = get_sw("stop_words_short.txt")
    used_words = get_list_from("used_words.txt")
    used_combs = get_list_from("used_combs.txt")
    
    w, c = parse_shamsi_events(sw)
    used_words[3] += w
    used_combs[3] += c
    
    used_words = [list(set(w)) for w in used_words]
    used_combs = [list(set(w)) for w in used_combs]
    
    return used_words, used_combs

In [43]:
def score(sent: str, c: int) -> int:
    ret = 0
    for comb in combs[c]:
        if comb in sent:
            print('a', comb)
            ret += COMB_SCORE
    for w in words[c]:
        if w in sent.split(' '):
            print(w, c)
            ret += WORD_SCORE
    return ret

In [44]:
def total_score(sent: str, c: int) -> dict:
    scoring = {}
    for i in range(4):
        scoring[i] = score(sent, i)
    return scoring

In [45]:
def return_scores (sent) : 
    norm = Normalizer()
    c = -1
    print(norm.normalize(sent))
    return total_score(norm.normalize(sent), c)